In [ ]:
## SALIENCY MAPS ON PRE-TRAINED MODELS

# 📚 bring in my libraries!
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.vgg16 import decode_predictions

import numpy as np

# our old friend ...
import matplotlib.pyplot as plt

# check that tensorflow installed correctly
print('tensorflow {}'.format(tf.__version__))



In [ ]:
# VGG16 is a pre-trained model, available via tensorflow
# VGG16 is a neural network architecture particularly good @ image classification
# keras = the API that talks to tensorflow

model = keras.applications.VGG16(weights='imagenet')

In [ ]:
model.summary()

# gives me the summary of the model we've downloaded (VGG16)
# check it out: there are 16 "layers" to the model
# each layer in a neural network represents a series of operations
# performed on the input data as it propagates through the network
# on its way to producing the final output

In [ ]:
# load in an image!
# the dimensions here (224x224 pixels) need to match what the pre-trained model expects

_img = keras.preprocessing.image.load_img('/content/cat_sq.jpg',target_size=(224,224))
plt.imshow(_img)
plt.show()


In [ ]:
# preprocess image to get it into the right format for the model

img = keras.preprocessing.image.img_to_array(_img)
# this line converts to image to a NumPy array of pixels
# x, y, and RGB value - for every pixel (224 x 224 x 3)

img = img.reshape((1, *img.shape))
# more preprocessing, adding a dimension ("batch") that the model expects
# a 4D array: (1 x 224 x 224 x 3)


y_pred = model.predict(img)
print(y_pred)
# an output of predictions from the image classification model
# each # is a "score" of how much the image is likely to be a certain class
# https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
# 1000 possible classes

In [ ]:
# this loops through the top 5 highest predicions in y_pred
# and links them to the associated categories (decode_predictions function is built-in)
# this prints out what the model "thinks" the image is
decoded_predictions = decode_predictions(y_pred, top=5)[0]
for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
    print(f"{i + 1}: {label} ({score:.2f})")


In [ ]:
# BIG PICTURE: we are trying to get a measure of which PARTS of the image
# are most important, predictive features for the model

# gradient here is a measure of CHANGE: how much will the output change if you adjust the input?
# remember that gradient is looking to minimize the LOSS function

# convert input image to tf.Variable, a multi-dimensional array for the model
images = tf.Variable(img, dtype=float)

# gradientTape() keeps track of all these gradients
# we're looking for the most likely class again (just in a different format)
with tf.GradientTape() as tape:
    pred = model(images, training=False)
    class_idxs_sorted = np.argsort(pred.numpy().flatten())[::-1]
    loss = pred[0][class_idxs_sorted[0]]

grads = tape.gradient(loss, images)


In [ ]:
# get absolute value
dgrad_abs = tf.math.abs(grads)

In [ ]:
# looking for the maximum in the list of absolute gradient values
# highest gradient = most sensitive to change
dgrad_max_ = np.max(dgrad_abs, axis=3)[0]

In [ ]:
# normalize to range between 0 and 1
arr_min, arr_max  = np.min(dgrad_max_), np.max(dgrad_max_)
grad_eval = (dgrad_max_ - arr_min) / (arr_max - arr_min + 1e-18)


In [ ]:
# NOW! we plot

# hello again, matplotlib ...
fig, axes = plt.subplots(1,2,figsize=(14,5))

# the first subplot = the input image
axes[0].imshow(_img)

# the second subplot is the SALIENCY MAP:
# MOST IMPORTANT pixels highlighted for the models prediction!
i = axes[1].imshow(grad_eval,cmap="viridis",alpha=0.8)
fig.colorbar(i)


In [ ]:
model.summary()

In [ ]:
# we can also show ACTIVATION MAP:
# what the process looks like for every "layer" in the model

# make a list of all the model's layers
layer_names = [layer.name for layer in model.layers]

# loop through each layer and visualize the activation map
for layer_name in layer_names:
    # create a model that outputs the activation map for the current layer
    activation_model = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

    # get the activation map for the input image
    activations = activation_model.predict(img)
    print(activations.shape)

    if len(activations.shape) == 4:
      # display the activation map if layer is visual
      plt.figure()
      plt.imshow(activations[0, :, :, 0], cmap='viridis')
      plt.title(f'Activation Map for Layer: {layer_name}')
      plt.colorbar()

plt.show()
# show 'em all!


📸 Citations:

- https://usmanr149.github.io/urmlblog/cnn/2020/05/01/Salincy-Maps.html - (much of the code/idea for this demo comes from this post!)
- https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a (image classification categories)
- https://towardsdatascience.com/saliency-map-using-pytorch-68270fe45e80

In [ ]:
## BELOW: code for just 1 layer's activation map

# Define the layer for which you want to visualize the activation map
layer_name = 'block3_conv1'  # Change this to the desired layer

# Create a model that outputs the activation map for the specified layer
activation_model = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

# Get the activation map for the input image
activations = activation_model.predict(img)

# Visualize the activation map
plt.imshow(activations[0, :, :, 0], cmap='viridis')
plt.title(f'Activation Map for Layer: {layer_name}')
plt.colorbar()
plt.show()